<a href="https://colab.research.google.com/github/bodong408/Machine-Learning/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading Toronto Neighborhood Data us Pandas

In [35]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)


DataFrame 0 is the main table. Filter out rows Borough = 'Not assigned' and 'NaN'

In [36]:
df1 = df[0].loc[(df[0]['Borough'] != 'Not assigned') & (df[0]['Borough'] != 'NaN')]


In [37]:
df1.shape

(103, 3)

In [38]:
df2 = pd.read_csv('https://cocl.us/Geospatial_data')

In [42]:
df3 = pd.merge(df1, df2, left_on='Postal Code',right_on='Postal Code',how='outer',suffixes=('_left','_right'))
df3

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
